# Programación avanzada- UNICABA

## Ejercicio de cierre de unidad 2

### Analisis de trends musicales en streaming segun Billboard

fuente

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# dataframe
df = pd.read_csv(r"archivos\streaming_songs.csv")
df.head(10)

,Date,Song,Artist,Rank,Last Week,Peak Position,Weeks in Charts
0,23/01/2013,Thrift Shop,Macklemore & Ryan Lewis Featuring Wanz,1,1,1,-
1,23/01/2013,Locked Out Of Heaven,Bruno Mars,2,2,1,-
2,23/01/2013,Diamonds,Rihanna,3,3,1,-
3,23/01/2013,Ho Hey,The Lumineers,4,4,1,-
4,23/01/2013,It's Time,Imagine Dragons,5,5,1,-
5,23/01/2013,Don't You Worry Child,Swedish House Mafia Featuring John Martin,6,6,1,-
6,23/01/2013,Radioactive,Imagine Dragons,7,7,1,-
7,23/01/2013,Swimming Pools (Drank),Kendrick Lamar,8,8,1,-
8,23/01/2013,Scream & Shout,will.i.am & Britney Spears,9,9,1,-
9,23/01/2013,F**kin Problems,A$AP Rocky Featuring Drake| 2 Chainz & Kendric...,10,10,1,-


In [ ]:
# encontrar vacios
df.isnull().sum()
df.info()
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33050 entries, 0 to 33049
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             33050 non-null  object
 1   Song             33050 non-null  object
 2   Artist           33050 non-null  object
 3   Rank             33050 non-null  int64 
 4   Last Week        33050 non-null  int64 
 5   Peak Position    33050 non-null  int64 
 6   Weeks in Charts  33050 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.8+ MB
Filas: 33050, Columnas: 7


In [ ]:
df.describe()

,Rank,Last Week,Peak Position
count,33050.000000,33050.000000,33050.000000
mean,25.500000,23.190439,10.174554
std,14.431088,14.032791,11.959740
min,1.000000,1.000000,1.000000
25%,13.000000,11.000000,1.000000
50%,25.500000,22.000000,5.000000
75%,38.000000,35.000000,15.000000
max,50.000000,50.000000,148.000000


Los datos fueron recolectados desde el 23/01/2013 hasta el 24/09/2025, contiene 33050 registros de canciones de streaming calculadas por Billboard

In [5]:
# columna Date a datetime
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")

# Extraer el año
df["Year"] = df["Date"].dt.year

# canciones por año

songs_per_year = df.groupby("Year")["Song"].nunique().reset_index()
songs_per_year.columns = ["Year", "UniqueSongCount"]

average_unique_songs_per_year = songs_per_year["UniqueSongCount"].mean()
print(f"Promedio de canciones únicas por año: {average_unique_songs_per_year:.2f}")

Promedio de canciones únicas por año: 363.46


In [ ]:
# Graficar
fig = px.line(
    songs_per_year,
    x="Year",
    y="UniqueSongCount",
    markers=True,
    title="Evolución de canciones por año",
    labels={"Year": "Año", "UniqueSongCount": "Cantidad de canciones"},
)

fig.update_layout(template="plotly_white")
fig.show()

Hay una media de 343 canciones por año, se evidencia una mayor cantidad de canciones en streaming a medida que pasa el tiempo. Nota: la cantidad en el 2025 se debe a que solo se incluye hasta septiembre

In [ ]:
# canciones únicas
unique_songs = df["Song"].nunique()

#  card
fig = go.Figure(
    go.Indicator(
        mode="number",
        value=unique_songs,
        title={"text": " Canciones únicas en streaming"},
        number={"valueformat": ","},
    )
)

fig.update_layout(height=200, margin=dict(t=30, b=0, l=0, r=0), template="plotly_white")

fig.show()

In [ ]:
#  artistas únicos
unique_artists = df["Artist"].nunique()

#  card
fig = go.Figure(
    go.Indicator(
        mode="number",
        value=unique_artists,
        title={"text": " Artistas únicos"},
        number={"valueformat": ","},
    )
)

fig.update_layout(height=200, margin=dict(t=30, b=0, l=0, r=0), template="plotly_white")

fig.show()

In [ ]:
# Filtrar solo las filas donde Peak Position es 1
top_hits = df[df["Peak Position"] == 1]

#  #1 de cada artista
artist_counts = top_hits["Artist"].value_counts()

# artista con más #1
top_artist = artist_counts.idxmax()
top_count = artist_counts.max()
print(f"El artista con más semanas en el puesto #1 es: {top_artist} ({top_count})")
print(artist_counts.head(10))

El artista con más semanas en el puesto #1 es: Taylor Swift (348)
Artist
Taylor Swift              348
Drake                     303
Morgan Wallen             231
The Weeknd                170
Kendrick Lamar            166
Ariana Grande             155
Miley Cyrus               148
PSY                       123
Olivia Rodrigo            119
Post Malone & Swae Lee    117
Name: count, dtype: int64


In [ ]:
# top 10 artistas
top_10_artists = artist_counts.head(10).index.tolist()

# Filtrar los datos de esos artistas
top_10_data = top_hits[top_hits["Artist"].isin(top_10_artists)]

# Agrupar por año y artista
artist_year_counts = (
    top_10_data.groupby(["Year", "Artist"]).size().reset_index(name="NumberOneHits")
)
fig = px.bar(
    artist_year_counts,
    x="Year",
    y="NumberOneHits",
    color="Artist",
    barmode="group",
    title=" Evolución anual de los top 10 artistas con canciones en el puesto #1",
    labels={"Year": "Año", "NumberOneHits": "#1 por año"},
)

fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5),
    template="plotly_white",
    height=600,
)

fig.show()

In [ ]:
#  top 10 artistas
top_10_artists = artist_counts.head(5).index.tolist()

# Filtrar solo los datos de esos artistas
top_10_data = top_hits[top_hits["Artist"].isin(top_10_artists)]

# Agrupar por año y artista
artist_year_counts = (
    top_10_data.groupby(["Year", "Artist"]).size().reset_index(name="NumberOneHits")
)
import plotly.express as px

fig = px.line(
    artist_year_counts,
    x="Year",
    y="NumberOneHits",
    color="Artist",
    markers=True,
    title=" Evolución temporal de los top 10 artistas con canciones en el puesto #1",
    labels={"Year": "Año", "NumberOneHits": "#1 por año"},
)

fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5),
    template="plotly_white",
    height=600,
)

fig.show()

In [ ]:
top_1 = df[df["Peak Position"] == 1].copy()
# Extraer el año
top_1["year"] = top_1["Date"].dt.year

# Contar artistas únicos por año
artistas_por_anio = top_1.groupby("Year")["Artist"].nunique().reset_index()
artistas_por_anio.columns = ["año", "artistas_unicos_en_posicion_1"]

print(artistas_por_anio)

fig = px.line(
    artistas_por_anio,
    x="año",
    y="artistas_unicos_en_posicion_1",
    markers=True,
    title="Artistas únicos en la posición #1 por año",
    labels={"anio": "Año", "artistas_unicos_en_posicion_1": "Artistas únicos en #1"},
)

fig.update_layout(
    xaxis=dict(dtick=1), yaxis=dict(title="Cantidad de artistas"), hovermode="x unified"
)

fig.show()

     año  artistas_unicos_en_posicion_1
0   2013                            182
1   2014                            140
2   2015                            124
3   2016                            144
4   2017                            166
5   2018                            200
6   2019                            198
7   2020                            253
8   2021                            239
9   2022                            229
10  2023                            197
11  2024                            199
12  2025                            132


In [ ]:
# mejor posición que alcanzó cada canción
best_positions = df.groupby("Song", as_index=False)["Peak Position"].min()

# Unir con los artistas
song_artist = df.groupby("Song", as_index=False)["Artist"].first()

# Merge para tener artista + mejor posición
merged = pd.merge(best_positions, song_artist, on="Song")

# Filtrar canciones que llegaron al top 10
top_10_songs = merged[merged["Peak Position"] <= 10]

#  artistas únicos que llegaron al top 10
top_10_artists = top_10_songs["Artist"].unique()

# Contar cuántos artistas son
num_top_10_artists = len(top_10_artists)

print(f" Total de artistas que llegaron alguna vez al top 10: {num_top_10_artists}")

 Total de artistas que llegaron alguna vez al top 10: 1705
